In [1]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import shutil
import time
import datetime as dt
import itertools
import re
from collections import Counter
from collections import defaultdict
import operator
import math
import functools
import collections
import random
from matplotlib.colors import DivergingNorm

#0-1 normalisation
from sklearn import preprocessing

from scipy.stats import pearsonr
import scipy.stats as stats

#modelling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, cohen_kappa_score, r2_score,\
mean_squared_error, mean_absolute_error, explained_variance_score#catboost, for a better support of categorical data
from catboost import CatBoostClassifier, Pool

#plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import cleaning_processing, HenDailyVariable_Origins
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
random.seed(0)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
#initialise parameters
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
dico_matching = config.dico_matching
dico_night_hour = config.dico_night_hour
nbr_maxdur2beremoved = config.nbr_maxdur2beremoved
dico_garden_opening_hour = config.dico_garden_opening_hour
path_extracted_data_cleaning = os.path.join(path_extracted_data,'Cleaning')
path_extracted_data_cleaning_model = os.path.join(path_extracted_data_cleaning,'model')  
path_extracted_data_cleaning_rawvscleaned = os.path.join(path_extracted_data_cleaning,'Raw&Cleaned')  
#create a director if not existing
if not os.path.exists(path_extracted_data_cleaning_rawvscleaned):
    os.makedirs(path_extracted_data_cleaning_rawvscleaned)
    
path_extracted_data_cleaning_rawvscleaned_DF = os.path.join(path_extracted_data_cleaning_rawvscleaned,'var_expl')
if not os.path.exists(path_extracted_data_cleaning_rawvscleaned_DF):
    os.makedirs(path_extracted_data_cleaning_rawvscleaned_DF)

#feature for model
li_cont_select = config.li_cont_select
li_bin = config.li_bin
li_cat_select = config.li_cat_select

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Download data & model

In [5]:
#start end of october, on data that the system never saw!!
date_min = dt.datetime(2020, 10, 27)
#end at the very end to have most info
date_max = dt.datetime(2022,12,30)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
#download data and add the necessary processing as the duraiton, next/previous zones etc
START_TIME = time.perf_counter()
#min date is dt.datetime(2020, 10, 15) so that we dont have only birds not moving
df = cleaning_processing(date_min=date_min, date_max=date_max, config=config)
df['time'] = df['Timestamp'].map(lambda x: dt.datetime.time(x))
df.rename(columns={'Zone':'Trackingsystem_Zone'}, inplace=True)
df.to_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned,'df_records_initial_processed.csv'), sep=';', index=False)
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\camil\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(2425329, 15)


  0%|                                                                                          | 0/179 [00:00<?, ?it/s]

(2148188, 15)


100%|████████████████████████████████████████████████████████████████████████████████| 179/179 [03:35<00:00,  1.21s/it]


...

,system,zone2,zone3,zone4,zone2_match,zone3_match,zone4_match
277141,3 - 5,2,1,0,3_Zone,2_Zone,NoOtherZone
277142,3 - 5,3,0,0,4_Zone,NoOtherZone,NoOtherZone
277143,3 - 5,4,0,0,5_Zone,NoOtherZone,NoOtherZone
277144,3 - 5,3,4,0,4_Zone,5_Zone,NoOtherZone
277145,3 - 5,4,0,0,5_Zone,NoOtherZone,NoOtherZone
277146,3 - 5,3,4,0,4_Zone,5_Zone,NoOtherZone
277147,3 - 5,3,1,0,4_Zone,2_Zone,NoOtherZone
277148,3 - 5,2,3,1,3_Zone,4_Zone,2_Zone
277149,8 - 9,1,3,6,2_Zone,4_Zone,miniZone3
277150,8 - 9,4,3,1,5_Zone,4_Zone,2_Zone


specifically, a small verification of when zone2 was equal to 9


,Timestamp,HenID,Zone,PenID,TagID,signalstrength,system,date,zone2,signalstzone2,...,next3_zone,previous3_zone,next4_zone,previous4_zone,next5_zone,previous5_zone,previousZone==NextZone,zone2_match,zone3_match,zone4_match
278062,2020-10-27 05:14:29,hen_47,2_Zone,pen12,tag_175,6,10 - 12,2020-10-27,9,4,...,2_Zone,3_Zone,2_Zone,NaN,2_Zone,NaN,True,2_Zone,5_Zone,3_Zone
278072,2020-10-27 05:14:45,hen_68,2_Zone,pen12,tag_179,6,10 - 12,2020-10-27,9,5,...,3_Zone,3_Zone,3_Zone,3_Zone,3_Zone,3_Zone,True,2_Zone,NoOtherZone,NoOtherZone
278109,2020-10-27 05:16:02,hen_68,2_Zone,pen12,tag_179,18,10 - 12,2020-10-27,9,8,...,3_Zone,2_Zone,3_Zone,2_Zone,3_Zone,3_Zone,False,2_Zone,5_Zone,3_Zone


Total running time: 25.12 mn
(2144152, 40)


,Timestamp,HenID,Trackingsystem_Zone,PenID,TagID,signalstrength,system,date,signalstzone2,next1_record_date,...,next5_zone,previous5_zone,previousZone==NextZone,zone2_match,zone3_match,zone4_match,zone3_match_exist,next2zone==Zone,previous2zone==Zone,time
277141,2020-10-27 00:03:07,hen_100,4_Zone,pen5,tag_121,6,3 - 5,2020-10-27,5,2020-10-27 00:03:12,...,5_Zone,NaN,False,3_Zone,2_Zone,NoOtherZone,1,0,0,00:03:07
277142,2020-10-27 00:03:12,hen_100,5_Zone,pen5,tag_121,9,3 - 5,2020-10-27,6,2020-10-27 00:20:52,...,3_Zone,NaN,False,4_Zone,NoOtherZone,NoOtherZone,0,1,0,00:03:12
277143,2020-10-27 00:15:39,hen_73,4_Zone,pen4,tag_41,4,3 - 5,2020-10-27,3,2020-10-27 00:15:42,...,3_Zone,NaN,False,5_Zone,NoOtherZone,NoOtherZone,0,1,0,00:15:39


In [7]:
model = CatBoostClassifier()
model.load_model(os.path.join(path_extracted_data_cleaning_model,'FINAL_Catboost'));

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Data preprocessing

In [5]:
df = pd.read_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned,'df_records_initial_processed.csv'), sep=';',
                 parse_dates=['Timestamp','date'], dayfirst=True) 
df['is_dur_bigger_'+str(nbr_maxdur2beremoved)] = df['duration'].map(lambda x: x>=nbr_maxdur2beremoved)
display(df['is_dur_bigger_'+str(nbr_maxdur2beremoved)].value_counts())
df['is_dur_smaller_60sec'] = df['duration'].map(lambda x: x<60)
display(df['is_dur_smaller_60sec'].value_counts())
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


6.0


False    1960341
True      183811
Name: is_dur_bigger_900, dtype: int64

False    1081483
True     1062669
Name: is_dur_smaller_60sec, dtype: int64

(2144152, 42)


,Timestamp,HenID,Trackingsystem_Zone,PenID,TagID,signalstrength,system,date,signalstzone2,next1_record_date,...,previousZone==NextZone,zone2_match,zone3_match,zone4_match,zone3_match_exist,next2zone==Zone,previous2zone==Zone,time,is_dur_bigger_900,is_dur_smaller_60sec
0,2020-10-27 00:03:07,hen_100,4_Zone,pen5,tag_121,6,3 - 5,2020-10-27,5,2020-10-27 00:03:12,...,False,3_Zone,2_Zone,NoOtherZone,1,0,0,00:03:07,False,True
1,2020-10-27 00:03:12,hen_100,5_Zone,pen5,tag_121,9,3 - 5,2020-10-27,6,2020-10-27 00:20:52,...,False,4_Zone,NoOtherZone,NoOtherZone,0,1,0,00:03:12,True,False
2,2020-10-27 00:15:39,hen_73,4_Zone,pen4,tag_41,4,3 - 5,2020-10-27,3,2020-10-27 00:15:42,...,False,5_Zone,NoOtherZone,NoOtherZone,0,1,0,00:15:39,False,True


### handle nan

In [9]:
#add info on nan values in the initial df, in order to replace at the end all prediction based on features involvig at least 
#one nan, by 1
#Indeed, for records with some nan features, we will predict them a 1 all the time its only 0,05% of the time , i.e. when its 
#first or last transition. Indeed, we dont want to loose any records!
df['nbr_nan'] = df.filter(li_cont_select+li_cat_select, axis=1).isnull().sum(axis=1).tolist()
display(df['nbr_nan'].value_counts())

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    2143531
1        443
2        178
Name: nbr_nan, dtype: int64

In [10]:
#fillnan for simplicity, but we will remove those rows predictions anyway
#display BEFORE categoricalfeatures
display(df[li_cat_select].head(3))
df[li_cont_select] = df[li_cont_select].fillna(df[li_cont_select].mean().iloc[0])
#display AFTER categoricalfeatures
df[li_cat_select] = df[li_cat_select].fillna(df[li_cat_select].mode().iloc[0])
df[li_cat_select].head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Trackingsystem_Zone,zone2_match,previous1_zone,next1_zone,system
0,4_Zone,3_Zone,NaN,5_Zone,3 - 5
1,5_Zone,4_Zone,4_Zone,3_Zone,3 - 5
2,4_Zone,5_Zone,NaN,3_Zone,3 - 5


,Trackingsystem_Zone,zone2_match,previous1_zone,next1_zone,system
0,4_Zone,3_Zone,3_Zone,5_Zone,3 - 5
1,5_Zone,4_Zone,4_Zone,3_Zone,3 - 5
2,4_Zone,5_Zone,3_Zone,3_Zone,3 - 5


### scaling

In [11]:
#scaling numerical data: necessary due to duration high values
display(df[li_cont_select].head(3))
sc = StandardScaler()
li_scale = [x for x in li_cont_select if x not in li_bin]
df[li_scale] = sc.fit_transform(df[li_scale])
display(df[li_cont_select].head(3))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,signalstrength,signalstzone2,duration,next_duration,previous_duration,next2zone==Zone,previous2zone==Zone,zone3_match_exist
0,6,5,5.0,1060.0,0.0,0,0,1
1,9,6,1060.0,7.0,5.0,1,0,0
2,4,3,3.0,297.0,0.0,1,0,0


,signalstrength,signalstzone2,duration,next_duration,previous_duration,next2zone==Zone,previous2zone==Zone,zone3_match_exist
0,-1.344953,-0.737476,-0.123721,0.123503,-0.124897,0,0,1
1,-0.595354,-0.465135,0.123292,-0.123163,-0.123727,1,0,0
2,-1.844685,-1.282158,-0.124189,-0.055230,-0.124897,1,0,0


In [12]:
df[li_cat_select].head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Trackingsystem_Zone,zone2_match,previous1_zone,next1_zone,system
0,4_Zone,3_Zone,3_Zone,5_Zone,3 - 5
1,5_Zone,4_Zone,4_Zone,3_Zone,3 - 5
2,4_Zone,5_Zone,3_Zone,3_Zone,3 - 5


# Cleaning

In [13]:
####################### predict
START_TIME = time.perf_counter()
#predict, and if its wintergarten (i.e. 1_Zone) put always = 1
df_ = df[df['Trackingsystem_Zone']!='1_Zone'].copy()
predictions = model.predict(df_.filter(li_cont_select+li_cat_select, axis=1))
df_['model_prediction'] = predictions
df = pd.merge(df, df_.filter(['model_prediction','HenID','Timestamp'], axis=1), on=['HenID','Timestamp'], how='left')

#predict false if <60seconds in WG else True
df.loc[df['Trackingsystem_Zone']=='1_Zone','model_prediction'] = 1
df.loc[(df['is_dur_smaller_60sec'])&(df['Trackingsystem_Zone']=='1_Zone'), 'model_prediction'] = 0

#always =1 if >15mn or if there was some nan values
df.loc[(df['is_dur_bigger_'+str(nbr_maxdur2beremoved)])|(df['nbr_nan']!=0), 'model_prediction'] = 1

#prediction_proba = model.predict_proba(df[df['Trackingsystem_Zone']!='1_Zone'])
#df['model_prediction_proba0'] = [round(x[0],2) for x in prediction_proba]
#df['model_prediction_proba1'] = [round(x[1],2) for x in prediction_proba]
END_TIME = time.perf_counter()
print ("Total running time for prediction: %.2f mn" %((END_TIME-START_TIME)/60))
display(df['model_prediction'].value_counts(normalize=True)) #in testing it was 4,9% of wrong records: 42/(42+821)*100)
#will take TIME to save
df.filter(['HenID','PenID','system','Trackingsystem_Zone','model_prediction','Timestamp','date'],
          axis=1).to_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned, id_run+'_Rawdata&Model.csv'),
                         index=False, sep=';')

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Total running time for prediction: 0.20 mn


1.0    0.947516
0.0    0.052484
Name: model_prediction, dtype: float64

####################### add the induced model_Observation (so that we can easily compare in the same excel as the raw data)
'''df = df.sort_values(['Timestamp'], ascending=True)
df_ = df[df['model_prediction']==0].copy()
df_['previous1_correctzone'] = df_.apply(lambda x: ([None]+list(df[(df['HenID']==x['HenID'])&\
                                                                (df['Timestamp']<x['Timestamp'])&\
                                                (df['model_prediction']==1)]['Trackingsystem_Zone'].values))[-1], axis=1)

df_['previous1_correctzone'] = df_.apply(lambda x: df[(df['HenID']==x['HenID'])&(df['Timestamp']<x['Timestamp'])&\
                                                    (df['model_prediction']==1)]['Trackingsystem_Zone'], axis=1)
df = pd.merge(df, df_.filter(['previous1_correctzone','HenID','Timestamp'], axis=1), on=['HenID','Timestamp'], how='left')

#add the zone that would be selected with this algorithm output: if==0, then the true zone is the one before
df['model_ObservedZone'] = df.apply(lambda x: x['Trackingsystem_Zone'] if x['model_prediction']==1 else \
                                    x['previous1_correctzone'],axis=1)'''
display(df.head(3))
#total time
END_TIME = time.perf_counter()
print ("Total running time to process&save prediction: %.2f mn" %((END_TIME-START_TIME)/60))


# Assess henID/system/pen explained variance of match vs non-match

li_sel_hen = list(df['HenID'].unique())
#restrict to 50 hens:
#li_sel_hen = random.sample(list(df['HenID'].unique()),50) #not enough 50 to assess penID and system explained variance
#number of disjoint set of days to run per #days involved
nbr_comb = 10
#compute disjoint set of days 
li_disjoint_set = []
#min nbr of days to include
min_days = 4

#to have disjoint set of days
li_date = df['date'].unique()
for nbr_days in tqdm.tqdm(range(min_days, int(len(li_date)/nbr_comb)+1)):
    li_sel = []
    li_date_ = li_date.copy()
    while len(li_sel)!=nbr_comb:
        li_ = random.sample(list(li_date_), nbr_days)
        li_sel.append(li_)
        li_date_ = [x for x in li_date_ if x not in li_]
    li_disjoint_set.extend(li_sel)
#do the one with all batches for final outputs
len(li_disjoint_set)
#li_disjoint_set

for i,li_d in tqdm.tqdm(enumerate(li_disjoint_set)):
    nbr_days = len(li_d)
    df_ = df[(df['date'].isin(li_d)) &(df['HenID'].isin(li_sel_hen))].copy()
    df_ = df_.dropna()
    #save to do in R
    df_.filter(['HenID', 'PenID', 'system','Zone','model_prediction'],
               axis=1).to_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned_DF,
                                           'df_EV_day'+str(nbr_days)+'_'+str(i)+'.csv'), sep=';', index=False)

df_varexp = pd.read_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned,'var_expl','df_VarExpl.csv'),
                       names=['csvID', 'Deviance', 'indvar'], index_col=False, sep=';', header=None)
df_varexp = df_varexp.iloc[1:79]
df_varexp['nbr_days'] = df_varexp['csvID'].map(lambda x: int(x.split('day')[1].split('_')[0]))
df_varexp['Deviance'] = df_varexp['Deviance'].map(lambda x: np.round(float(x),2))
print(df_varexp.shape)
display(df_varexp.head(3))

#plot
ax = sns.boxplot(x="nbr_days", y="Deviance", hue='indvar', data=df_varexp)
#ax = sns.swarmplot(x="nbr_days", y="Deviance", hue='indvar', data=df_varexp)
#a réentrainer dans le temps? ou peut-etre que dans le temps les erreur entre poule se compenseront et limportance de la 
#henid diminuerai
plt.savefig(os.path.join(path_extracted_data_cleaning_rawvscleaned, 'deviance_features_overtime.png'), 

# Compute variable

In [5]:
#for being more reproductible, we open the file that was saved from cleaning
df = pd.read_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned, id_run+'_Rawdata&Model.csv'), sep=';', 
                 parse_dates=['Timestamp','date'], dayfirst=True)
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(2144152, 7)


,HenID,PenID,system,Trackingsystem_Zone,model_prediction,Timestamp,date
0,hen_100,pen5,3 - 5,4_Zone,1.0,2020-10-27 00:03:07,2020-10-27
1,hen_100,pen5,3 - 5,5_Zone,1.0,2020-10-27 00:03:12,2020-10-27
2,hen_73,pen4,3 - 5,4_Zone,1.0,2020-10-27 00:15:39,2020-10-27


In [6]:
#start end of october, on data that the system never saw!!
date_min = dt.datetime(2020, 10, 20)
#end at the very end to have most info
date_max = dt.datetime(2022, 11, 5)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
############## cleaned dataset
START_TIME = time.perf_counter()
df_ = df[(df['date']<date_max)&(df['date']>date_min)].copy()
#remove predicted-wrong-records
print(df_.shape)
df_ = df_[df_['model_prediction']!=0].copy()
print(df_.shape)
#simply rename the tracking system zone into the model zone :) 
df_.rename(columns={'Trackingsystem_Zone':'Zone'}, inplace=True)
display(df_.head(2))
for p in tqdm.tqdm(df_['PenID'].unique()):
    df_daily = HenDailyVariable_Origins(df_[df_['PenID']==p], config, name_='CLEANING_'+p, timestamp_name='Timestamp',
                                       has_cons_equal_zone=True)
    print(df_daily.shape)
    df_daily.head(3)
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))  

#Concatenate all HenVariables csv into one
li_paths_var = glob.glob(os.path.join(path_extracted_data, id_run+'_daily__CLEANING'+'*_variables.csv'))
li_df = []
for path_var in tqdm.tqdm(li_paths_var):
    #for being more reproductible, we open the file that was saved from cleaning
    df_ = pd.read_csv(path_var, sep=';',parse_dates=['level', 'FirstTimestamp_1_Zone', 'FirstTimestamp_2_Zone', 
                                                     'FirstTimestamp_3_Zone','FirstTimestamp_4_Zone',
                                                     'FirstTimestamp_5_Zone']) 
    print(df_.shape)
    li_df.append(df_)
df_daily = pd.concat(li_df)
df_daily.to_csv(os.path.join(path_extracted_data, id_run+'_daily_CLEANING_ALL_variables.csv'), sep=';', index=False)
print(df_daily.shape)
df_daily.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(2144152, 7)
(2031618, 7)


,HenID,PenID,system,Zone,model_prediction,Timestamp,date
0,hen_100,pen5,3 - 5,4_Zone,1.0,2020-10-27 00:03:07,2020-10-27
1,hen_100,pen5,3 - 5,5_Zone,1.0,2020-10-27 00:03:12,2020-10-27


  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

----------------- Create time serie
in this time series there is 23 hens
The initial starting date in over all is: 2020-10-27 00:03:07, and the ending date will be: 2021-01-18 08:36:57
But note that birds may have different ending and starting date which should be taken into account when computing variables
and after ending the last day at midnight : 2020-10-27 00:03:07, and the ending date will be: 2021-01-18 23:59:59
Total running time: 2.00 mn
your time series has 1 seconds between two timestamps
----------------- main night zone and nbr of transitions over night....
----------------- total duration per Zone in seconds....
----------------- first timestamp in each zone per day....
----------------- number of Zone (excluding nan)....
----------------- compute some variables based on a list of zones over a day....
----------------NESTBOX


...

(1736, 117)


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:19<00:00,  2.50s/it]

(1817, 117)


(13944, 117)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,dur_values,dur_values_normalized,...,list_of_night20_2_temperature,nbr_temperature_nnnan,temperature_night20_2_median,temperature_night20_2_max,temperature_night20_2_var,list_of_MovementCounter_day,len_MovementCounter_day,MovementCounter_day_amount_nnactivity,MovementCounter_day_max,MovementCounter_day_mean
0,hen_131,2020-10-27,2258.0,19888.0,16978.0,3204.0,872.0,43200.0,"[2258.0, 19888.0, 16978.0, 3204.0, 872.0]","[0.05226851851851852, 0.4603703703703704, 0.39...",...,"[29.0, 29.0, 30.0, 29.0, 29.0, 29.0, 29.0, 30....",36.0,29.0,30.0,0.212191,"[41.0, 62.0, 72.0, 71.0, 63.0, 50.0, 36.0, 39....",72.0,0.000000,103.0,67.236111
1,hen_131,2020-10-28,3643.0,19470.0,17180.0,2504.0,403.0,43200.0,"[3643.0, 19470.0, 17180.0, 2504.0, 403.0]","[0.0843287037037037, 0.45069444444444445, 0.39...",...,"[28.0, 28.0, 28.0, 28.0, 28.0, 29.0, 29.0, 28....",36.0,29.0,29.0,0.212191,"[45.0, 78.0, 59.0, 64.0, 52.0, 43.0, 43.0, 55....",72.0,0.000000,109.0,69.138889
2,hen_131,2020-10-29,2642.0,21300.0,19136.0,2727.0,995.0,46800.0,"[2642.0, 21300.0, 19136.0, 2727.0, 995.0]","[0.05645299145299145, 0.4551282051282051, 0.40...",...,"[29.0, 29.0, 29.0, 29.0, 29.0, 29.0, 29.0, 29....",36.0,29.0,29.0,0.076389,"[18.0, 30.0, 30.0, 65.0, 48.0, 44.0, 66.0, 61....",78.0,0.012821,108.0,69.333333


In [9]:
############## raw dataset
#open time series per session and compute the variables for each session (car time series make sence at session level), et en 
#plus des variables tel que running entropy over the whole session ateach last timestamp of each level make sence only at 
#session level
#once to much data, do this per pen!!! with name=pens!!
START_TIME = time.perf_counter()
df_ = df[(df['date']<date_max)&(df['date']>date_min)].copy()
df_.rename(columns={'Trackingsystem_Zone':'Zone'}, inplace=True)
display(df_.head(2))
for p in tqdm.tqdm(df_['PenID'].unique()):
    df_daily = HenDailyVariable_Origins(df_[df_['PenID']==p], config, name_='RAW_'+p, timestamp_name='Timestamp',
                                       has_cons_equal_zone=True)
    print(df_daily.shape)
    df_daily.head(3)
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))  

#Concatenate all HenVariables csv into one
li_paths_var = glob.glob(os.path.join(path_extracted_data, id_run+'_daily__RAW'+'*_variables.csv'))
li_df = []
for path_var in tqdm.tqdm(li_paths_var):
    #for being more reproductible, we open the file that was saved from cleaning
    df_ = pd.read_csv(path_var, sep=';',parse_dates=['level', 'FirstTimestamp_1_Zone', 'FirstTimestamp_2_Zone', 
                                                     'FirstTimestamp_3_Zone','FirstTimestamp_4_Zone',
                                                     'FirstTimestamp_5_Zone']) 
    print(df_.shape)
    li_df.append(df_)
df_daily = pd.concat(li_df)
df_daily.to_csv(os.path.join(path_extracted_data, id_run+'_daily_RAW_ALL_variables.csv'), sep=';', index=False)
print(df_daily.shape)
df_daily.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,HenID,PenID,system,Zone,model_prediction,Timestamp,date
0,hen_100,pen5,3 - 5,4_Zone,1.0,2020-10-27 00:03:07,2020-10-27
1,hen_100,pen5,3 - 5,5_Zone,1.0,2020-10-27 00:03:12,2020-10-27


  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

----------------- Create time serie
in this time series there is 23 hens
The initial starting date in over all is: 2020-10-27 00:03:07, and the ending date will be: 2021-01-18 08:36:57
But note that birds may have different ending and starting date which should be taken into account when computing variables
and after ending the last day at midnight : 2020-10-27 00:03:07, and the ending date will be: 2021-01-18 23:59:59
Total running time: 1.43 mn
your time series has 1 seconds between two timestamps
----------------- main night zone and nbr of transitions over night....
----------------- total duration per Zone in seconds....
----------------- first timestamp in each zone per day....
----------------- number of Zone (excluding nan)....
----------------- compute some variables based on a list of zones over a day....
----------------NESTBOX



100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:08<00:00,  2.76it/s]
C:\Users\camil\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1366: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


...

(1736, 117)


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:21<00:00,  2.69s/it]

(1817, 117)


(13944, 117)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,dur_values,dur_values_normalized,...,list_of_night20_2_temperature,nbr_temperature_nnnan,temperature_night20_2_median,temperature_night20_2_max,temperature_night20_2_var,list_of_MovementCounter_day,len_MovementCounter_day,MovementCounter_day_amount_nnactivity,MovementCounter_day_max,MovementCounter_day_mean
0,hen_131,2020-10-27,2258.0,19862.0,17004.0,3204.0,872.0,43200.0,"[2258.0, 19862.0, 17004.0, 3204.0, 872.0]","[0.05226851851851852, 0.4597685185185185, 0.39...",...,"[29.0, 29.0, 30.0, 29.0, 29.0, 29.0, 29.0, 30....",36.0,29.0,30.0,0.212191,"[41.0, 62.0, 72.0, 71.0, 63.0, 50.0, 36.0, 39....",72.0,0.000000,103.0,67.236111
1,hen_131,2020-10-28,3669.0,19439.0,17185.0,2504.0,403.0,43200.0,"[3669.0, 19439.0, 17185.0, 2504.0, 403.0]","[0.08493055555555555, 0.4499768518518519, 0.39...",...,"[28.0, 28.0, 28.0, 28.0, 28.0, 29.0, 29.0, 28....",36.0,29.0,29.0,0.212191,"[45.0, 78.0, 59.0, 64.0, 52.0, 43.0, 43.0, 55....",72.0,0.000000,109.0,69.138889
2,hen_131,2020-10-29,2741.0,21189.0,19148.0,2727.0,995.0,46800.0,"[2741.0, 21189.0, 19148.0, 2727.0, 995.0]","[0.05856837606837607, 0.4527564102564103, 0.40...",...,"[29.0, 29.0, 29.0, 29.0, 29.0, 29.0, 29.0, 29....",36.0,29.0,29.0,0.076389,"[18.0, 30.0, 30.0, 65.0, 48.0, 44.0, 66.0, 61....",78.0,0.012821,108.0,69.333333


In [12]:
#for being more reproductible, we open the file that was saved from cleaning
df_c = pd.read_csv(os.path.join(path_extracted_data, id_run+'_daily_CLEANING_ALL_variables.csv'), sep=';', parse_dates=['level'],
                 dayfirst=True) 
df_c['type'] = 'cleaning'
df_r = pd.read_csv(os.path.join(path_extracted_data, id_run+'_daily_RAW_ALL_variables.csv'), sep=';', parse_dates=['level'],
                 dayfirst=True) 
df_r['type'] = 'raw'
df_var = pd.concat([df_c,df_r])
df_var.to_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned,'daily_ALL_variables.csv'), sep=';', index=False)
print(df_r.shape,df_c.shape,df_var.shape)
print(df_var.shape)
df_var.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(13944, 118) (13944, 118) (27888, 118)
(27888, 118)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,dur_values,dur_values_normalized,...,nbr_temperature_nnnan,temperature_night20_2_median,temperature_night20_2_max,temperature_night20_2_var,list_of_MovementCounter_day,len_MovementCounter_day,MovementCounter_day_amount_nnactivity,MovementCounter_day_max,MovementCounter_day_mean,type
0,hen_131,2020-10-27,2258.0,19888.0,16978.0,3204.0,872.0,43200.0,"[2258.0, 19888.0, 16978.0, 3204.0, 872.0]","[0.05226851851851852, 0.4603703703703704, 0.39...",...,36.0,29.0,30.0,0.212191,"[41.0, 62.0, 72.0, 71.0, 63.0, 50.0, 36.0, 39....",72.0,0.000000,103.0,67.236111,cleaning
1,hen_131,2020-10-28,3643.0,19470.0,17180.0,2504.0,403.0,43200.0,"[3643.0, 19470.0, 17180.0, 2504.0, 403.0]","[0.0843287037037037, 0.45069444444444445, 0.39...",...,36.0,29.0,29.0,0.212191,"[45.0, 78.0, 59.0, 64.0, 52.0, 43.0, 43.0, 55....",72.0,0.000000,109.0,69.138889,cleaning
2,hen_131,2020-10-29,2642.0,21300.0,19136.0,2727.0,995.0,46800.0,"[2642.0, 21300.0, 19136.0, 2727.0, 995.0]","[0.05645299145299145, 0.4551282051282051, 0.40...",...,36.0,29.0,29.0,0.076389,"[18.0, 30.0, 30.0, 65.0, 48.0, 44.0, 66.0, 61....",78.0,0.012821,108.0,69.333333,cleaning


In [13]:
min(df_var['level'].unique()), max(df_var['level'].unique())

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(numpy.datetime64('2020-10-26T00:00:00.000000000'),
 numpy.datetime64('2021-01-18T00:00:00.000000000'))

df_var.sort_values(['duration_5_Zone','HenID'], inplace=True)
df_var['in_WG_15mnAfterOpening'] = df_var['in_WG_15mnAfterOpening'].astype(float)
li_var = ['duration_5_Zone','duration_4_Zone','duration_3_Zone','duration_2_Zone', #duration variables
          'nbr_bouts_1_Zone','in_WG_15mnAfterOpening', #garden variables
          'night_Total_number_transition','Total_number_transition'] #transition variables
df_hm = df_var[li_var+['HenID','level','type']]
df_hm = df_hm[~df_hm.isna().any(axis=1)]
df_sel_hen = df_hm.groupby(['HenID'])['level'].count().reset_index()
nbr = max(df_sel_hen['level'].unique())
li_sel_hen = df_sel_hen[df_sel_hen['level']==max(df_sel_hen['level'].unique())]['HenID'].tolist()
print('We have %d hens with %d days having 0 nan'%(len(li_sel_hen),nbr))
df_hm = df_hm[df_hm['HenID'].isin(li_sel_hen)]
df_hm.head(3)